In [1]:
import sqlite3
import redis
import gzip
import re
import time

In [3]:
def create_connection(db_file):
    try:
        con = sqlite3.connect(db_file)
        return con
    except Error as e:
        print(e)
    
    return None

In [4]:
def create_table(con, create_table_sql):
    try:
        c = con.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [5]:
db = "I:\sqlite_db\Docs.db"
sql_create_docs_table = '''
CREATE TABLE IF NOT EXISTS docs (
    id integer PRIMARY KEY,
    path text NOT NULL
);
'''
con = create_connection(db)
if con is None:
    print("Error! cannot create the database connection.")
else:
    create_table(con, sql_create_docs_table)
con.close()

In [6]:
def parseWet(content):
    docs = content.split('WARC-Type: conversion\r\n')
    docs = docs[1:]
    return docs

In [7]:
def isEnglish(word):
    if not word:
        return False
    for c in word:
        if not ((ord(c) >= ord('a') and ord(c) <= ord('z')) or 
                (ord(c) >= ord('A') and ord(c) <= ord('Z')) or 
                (ord(c) >= ord('0') and ord(c) <= ord('9'))):
            return False
    return True

In [11]:
def insert2sqlite(c, start_idx, docs):
    for idx, doc in enumerate(docs):
        num_terms = 0
        if not idx % 10000:
            print(idx)
        tmp_post = {}
        lines = doc.split('\n')
        lines = [line.strip() for line in lines]
        for i, line in enumerate(lines):
            if line[:len('Content-Length')] == 'Content-Length':
                break
        for line_idx, line in enumerate(lines[i + 2:]):
            words = re.split('[,. :-=]', line)
            for pos, word in enumerate(words):
                if not isEnglish(word):
                    continue
                num_terms += 1
        if num_terms:
            # add to redis
            start_idx += 1
#             file_path = "I:/sqlite_db/docs/" + str(start_idx)
#             with open(file_path, 'w', errors='ignore') as f:
#                 f.write('\n'.join(lines[i + 2:-4]))
            # write to sqlite the path, key is the start_idx
            sql_create_docs_table = "INSERT INTO docs VALUES(?, ?);"
            try:
                c.execute(sql_create_docs_table, (start_idx, '\n'.join(lines[i + 2:-4])))
            except Error as e:
                print(e)
    return start_idx

In [13]:
start_time = time.time()
db = "I:\sqlite_db\Docs.db"
start_idx = 0
con = create_connection(db)
c = con.cursor()
c.execute("DELETE FROM docs;")
for i in range(50):
    filename = 'I:/commoncrawl/gzs/' + str(i) +'.gz'
    print(filename, start_idx)
    with open(filename, 'rb') as f:
        content = gzip.decompress(f.read())
        content = content.decode('utf-8')       
        docs = parseWet(content)
        start_idx = insert2sqlite(c, start_idx, docs)
        con.commit()
con.close()
end_time = time.time()
print("Adding all documents to sqlite3 database takes {:.2f} seconds".format(end_time - start_time))

I:/commoncrawl/gzs/0.gz 0
0
10000
20000
30000
I:/commoncrawl/gzs/1.gz 39315
0
10000
20000
30000
I:/commoncrawl/gzs/2.gz 78630
0
10000
20000
30000
I:/commoncrawl/gzs/3.gz 117945
0
10000
20000
30000
I:/commoncrawl/gzs/4.gz 157260
0
10000
20000
30000
I:/commoncrawl/gzs/5.gz 196575
0
10000
20000
30000
I:/commoncrawl/gzs/6.gz 235890
0
10000
20000
30000
I:/commoncrawl/gzs/7.gz 275205
0
10000
20000
30000
I:/commoncrawl/gzs/8.gz 314520
0
10000
20000
30000
I:/commoncrawl/gzs/9.gz 353835
0
10000
20000
30000
I:/commoncrawl/gzs/10.gz 393150
0
10000
20000
30000
I:/commoncrawl/gzs/11.gz 432465
0
10000
20000
30000
I:/commoncrawl/gzs/12.gz 471780
0
10000
20000
30000
I:/commoncrawl/gzs/13.gz 511095
0
10000
20000
30000
I:/commoncrawl/gzs/14.gz 550410
0
10000
20000
30000
I:/commoncrawl/gzs/15.gz 589725
0
10000
20000
30000
I:/commoncrawl/gzs/16.gz 629040
0
10000
20000
30000
I:/commoncrawl/gzs/17.gz 668355
0
10000
20000
30000
I:/commoncrawl/gzs/18.gz 707670
0
10000
20000
30000
I:/commoncrawl/gzs/19.gz 7469